# Beyond accuracy evaluation

## Notes

 * Diversity at 100 (div@100) : on prend seulement les 100 premiers elements du ranking et on calcul la diversité de ceux-ci en terme de topic, de TFIDF et de style. Donc la moyenne des pairwise distances $$diversity(R) = \frac{\sum_{i=1}^{|R|}\sum_{j=i+1}^{|R|} dist(R_i, R_j)}{\frac{{|R|}^2-{|R|}}{2}}$$
 On ne prend que les 100 premier car si on prennait les 1000, alors tous les modèles auraient la même diversité.
 * Novelty at 100 (nov@100) : pareil mais entre l'historique utilisateur et R.
 $$novelty(R, H) = \frac{\sum_{i=1}^{|R|}\sum_{j=1}^{|H|} dist(R_i, H_j)}{|R|.|H|}$$
 * Strict novelty at 100 (snov@100) : pareil mais on prend le min.
 $$strictnovelty(R, H) = \frac{\sum_{i=1}^{|R|} mindist(R_i, H)}{|R|}$$
 * Serendipity at 100 (ser@100) : the ratio of relevants items the evaluated model recommanded and the primitive model didn't recommand. With $R$ the recommendation list of the evaluated model, $P$ the recommendation list of the primitive model, $T$ the set of relevant items, and for cases where $T \setminus P \neq \emptyset$, we define the serendipity as:
 $$serendipity(R, P, T) = \frac{|R \cap (T \setminus P)|}{|T \setminus P|}$$
 Cases where $T \setminus P = \emptyset$ are not relevant because the primitive model already predicted all relevant items. Thus no model can be serendipe. These cases are not taken into account in the average for all user (+ TODO donner le % des cas $T \setminus P = \emptyset$).

## Commands

In [ ]:
# tipi 86
# killbill bacceval ; cd ~/twinews-logs ; jupython -o nohup-bacceval-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/evaluation/bacceval.ipynb

## Imports

In [ ]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""
isNotebook = '__file__' not in locals()
TEST = isNotebook

In [5]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from datastructuretools.cache import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.evaluation import metrics
from twinews.evaluation.utils import *
from twinews.models.genericutils import *
import time
import pymongo

## Init

In [3]:
logger = Logger(tmpDir('logs') + "/beyong-accuracyevaluation.log") if isNotebook else Logger("evaluation-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

In [ ]:
# Making the cache that is a dict-like object (url --> vector) keeping data until 2Go of free RAM:
genericCaches = dict()
newsCollection = None
def getter(key, logger=None, verbose=True):
    global newsCollection
    global genericCaches
    global genericFields
    if newsCollection is None:
        newsCollection = getNewsCollection(logger=logger, verbose=verbose)
    cacheKey, url = key
    field = genericFields[cacheKey]
    if cacheKey in genericCaches:
        genericCache = genericCaches[cacheKey]
    else:
        genericCache = getGenericCache(cacheKey, readOnly=True, logger=logger, verbose=verbose)
        genericCaches[cacheKey] = genericCache
    row = newsCollection.findOne({'url': url}, projection={field: True})
    theHash = objectToHash(row[field])
    return genericCache[theHash]

In [6]:
cache = Cache(getter, logger=logger)

serializabledict twinews-tfidf initialised.


In [ ]:
cache

In [7]:
a = dict()

In [8]:
a["e"] = 0

In [10]:
a[(1, 2)] =      1